### Install necessary libraries


In [38]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
! pip install instaloader matplotlib numpy


##### Load Session using sessionID to login 

In [40]:

import instaloader
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from datetime import datetime
from datetime import datetime, timedelta
 
def load_instagram_session(session_id):
    """
    Load Instagram session using a session ID provided by the user.
    """
    try:
        loader = instaloader.Instaloader()
        loader.context._session.cookies.set("sessionid", session_id)
        return loader
    except Exception as e:
        print(f"Error loading session: {e}")
        return None


####  Fetch and display basic Instagram profile details 

In [41]:
def fetch_profile_data(username, loader):
   
    try:
        profile = instaloader.Profile.from_username(loader.context, username)
        print(f"Profile Name: {profile.full_name}")
        print(f"Bio: {profile.biography}")
        print(f"Followers: {profile.followers}")
        print(f"Following: {profile.followees}")
        return profile
    except Exception as e:
        print(f"Error fetching profile data: {e}")
        return None


#### Calculate engagement rate based on likes, comments, and followers

In [42]:
def calculate_engagement_rate(likes, comments, followers):

    try:
        total_engagement = sum(likes) + sum(comments)
        engagement_rate = (total_engagement / (followers * len(likes))) * 100
        return engagement_rate
    except ZeroDivisionError:
        return 0

def analyze_engagement_rate(profile):
   
    try:
        followers = profile.followers
        likes, comments,post_dates = [], [], []

        for post in profile.get_posts():
            if(datetime.now()-post.date).days<=180:
                likes.append(post.likes)
                comments.append(post.comments)
                post_dates.append(post.date.strftime('%Y-%m'))
            else:
                break
        # Calculate monthly engagement rates
        engagement_rates = {}
        for date, like, comment in zip(post_dates, likes, comments):
            if date not in engagement_rates:
                engagement_rates[date] = {'likes': 0, 'comments': 0, 'count': 0}
            engagement_rates[date]['likes'] += like
            engagement_rates[date]['comments'] += comment
            engagement_rates[date]['count'] += 1
        
        # Compute monthly rates
        monthly_engagement_rate = {}
        for date, data in engagement_rates.items():
            total_engagement = data['likes'] + data['comments']
            monthly_engagement_rate[date] = (total_engagement / (followers * data['count'])) * 100
        
        # Plot engagement rate graph
        plot_engagement_rate(monthly_engagement_rate)
        return monthly_engagement_rate
    except Exception as e:
        print(f"Error calculating engagement rate: {e}")
        return None

def plot_engagement_rate(monthly_engagement_rate):
    """
    Plot a line graph of engagement rates over the last 6 months.
    """
    months = list(monthly_engagement_rate.keys())
    rates = list(monthly_engagement_rate.values())

    plt.figure(figsize=(10, 6))
    plt.plot(months, rates, marker='o', color='blue', label="Engagement Rate (%)")
    plt.xlabel("Month")
    plt.ylabel("Engagement Rate (%)")
    plt.title("Engagement Rate Over Last 6 Months")
    plt.grid()
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()



#### Hashtag Categories 

In [43]:
HASHTAG_CATEGORIES = {
    "fashion": ["fashion", "style", "ootd", "trend", "vintage", "designer"],
    "beauty": ["beauty", "makeup", "skincare", "cosmetics", "beautytips"],
    "fitness": ["fitness", "workout", "health", "gym", "exercise"],
    "travel": ["travel", "wanderlust", "adventure", "vacation", "destination"],
    "food": ["food", "foodie", "yum", "recipe", "cooking"],
}

def categorize_influencer_by_hashtags(profile):
    """
    Categorize an influencer based on the hashtags in their posts.
    """
    try:
        hashtags = []
        for post in profile.get_posts():
            hashtags.extend(post.caption_hashtags)
            if len(hashtags) >= 50:  # Limit to 50 posts
                break

        hashtag_counter = Counter(hashtags)
        category_count = {category: 0 for category in HASHTAG_CATEGORIES}

        for hashtag, count in hashtag_counter.items():
            for category, keywords in HASHTAG_CATEGORIES.items():
                if hashtag in keywords:
                    category_count[category] += count

        influencer_type = max(category_count, key=category_count.get)
        print(f"Categorized as a '{influencer_type}' influencer.")
        plot_hashtag_frequency(hashtag_counter)
    except Exception as e:
        print(f"Error during categorization: {e}")

def plot_hashtag_frequency(hashtag_counter):
    """
    Plot the frequency of top hashtags.
    """
    common_hashtags = hashtag_counter.most_common(10)
    hashtags, counts = zip(*common_hashtags)
    plt.figure(figsize=(10, 6))
    plt.barh(hashtags, counts, color='skyblue')
    plt.xlabel('Frequency')
    plt.title('Top 10 Most Frequent Hashtags')
    plt.gca().invert_yaxis()
    plt.show()


##### Follower Growth Rate Formula:

In [44]:
def calculate_growth_rate(profile):
    """
    Calculate the monthly growth rate of followers.
    """
    try:
        followers_data = []  # Stores (date, followers_count)
        for post in profile.get_posts():
            if (datetime.now() - post.date).days <= 180:  # Limit to last 6 months
                followers_data.append((post.date, profile.followers))
            else:
                break
        
        # Sort by date (optional if already sorted)
        followers_data.sort(key=lambda x: x[0])

        # Calculate growth rate
        monthly_growth_rate = {}
        for i in range(1, len(followers_data)):
            prev_date, prev_followers = followers_data[i - 1]
            curr_date, curr_followers = followers_data[i]
            days_diff = (curr_date - prev_date).days
            growth = ((curr_followers - prev_followers) / prev_followers) * 100 if prev_followers > 0 else 0
            monthly_growth_rate[curr_date.strftime('%Y-%m')] = growth / (days_diff / 30)

        # Plot the growth rate
        plot_growth_rate(monthly_growth_rate)
        return monthly_growth_rate
    except Exception as e:
        print(f"Error calculating growth rate: {e}")
        return None

def plot_growth_rate(monthly_growth_rate):
    """
    Plot a bar chart of the monthly growth rate.
    """
    months = list(monthly_growth_rate.keys())
    growth_rates = list(monthly_growth_rate.values())

    plt.figure(figsize=(10, 6))
    plt.bar(months, growth_rates, color='orange', alpha=0.7, label="Growth Rate (%)")
    plt.xlabel("Month")
    plt.ylabel("Growth Rate (%)")
    plt.title("Follower Growth Rate Over Last 6 Months")
    plt.grid(axis='y')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

##### Collaboration Rate:

In [45]:
def calculate_collaboration_rate(profile):
    """
    Calculate the collaboration rate based on sponsored posts and visualize with a pie chart.
    """
    try:
        total_posts = 0
        sponsored_posts = 0
        for post in profile.get_posts():
            if (datetime.now() - post.date).days <= 180:
                total_posts += 1
                if '#sponsored' in post.caption or '#ad' in post.caption:
                    sponsored_posts += 1
            else:
                break

        collaboration_rate = (sponsored_posts / total_posts) * 100 if total_posts > 0 else 0
        print(f"Collaboration Rate: {collaboration_rate:.2f}%")

        # Visualization with pie chart
        plot_collaboration_rate(sponsored_posts, total_posts - sponsored_posts)
        return collaboration_rate
    except Exception as e:
        print(f"Error calculating collaboration rate: {e}")
        return None

def plot_collaboration_rate(sponsored_posts, organic_posts):
    """
    Plot a pie chart of collaboration vs organic posts.
    """
    labels = ['Sponsored Posts', 'Organic Posts']
    sizes = [sponsored_posts, organic_posts]
    colors = ['gold', 'lightblue']
    explode = (0.1, 0)  # Emphasize the sponsored posts

    plt.figure(figsize=(8, 8))
    plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
            shadow=True, startangle=140)
    plt.title("Collaboration Rate (Sponsored vs Organic Posts)")
    plt.axis('equal')  # Equal aspect ratio ensures a circular pie chart
    plt.show()

#### Estimate earnings per post based on followers and engagement rate

In [46]:
def estimate_earnings(followers, engagement_rate):
    """
    Estimate earnings per post based on followers and engagement rate.
    """
    rate_per_1000 = 0.10  # $0.10 per 1,000 followers
    estimated_earnings = (followers * engagement_rate) / 1000 * rate_per_1000
    return estimated_earnings

def plot_estimated_earnings(monthly_engagement_rate, followers):
    """
    Plot a bar chart of estimated earnings over the last 6 months.
    """
    months = list(monthly_engagement_rate.keys())
    earnings = [estimate_earnings(followers, rate) for rate in monthly_engagement_rate.values()]

    plt.figure(figsize=(10, 6))
    plt.bar(months, earnings, color='green', alpha=0.7, label="Earnings ($)")
    plt.xlabel("Month")
    plt.ylabel("Estimated Earnings ($)")
    plt.title("Estimated Earnings Per Month")
    plt.grid(axis='y')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()



#### Function to analyze an Instagram Profile 

In [ ]:
def instagram_analysis():

    session_id = input("Enter your Instagram session ID: ")
    username = input("Enter the Instagram username to analyze: ")
    
    loader = load_instagram_session(session_id)
    if not loader:
        return
    
    profile = fetch_profile_data(username, loader)
    if not profile:
        return
    
    print("\n--- Hashtag Categorization ---")
    categorize_influencer_by_hashtags(profile)
    
    print("\n--- Engagement Rate Analysis ---")
    monthly_engagement_rate = analyze_engagement_rate(profile)
    
    print("\n--- Estimated Earnings ---")
    if monthly_engagement_rate:
        plot_estimated_earnings(monthly_engagement_rate, profile.followers)
    print("\n--- Growth Rate Analysis ---")
    monthly_growth_rate = calculate_growth_rate(profile)

    print("\n--- Collaboration Rate ---")
    collaboration_rate = calculate_collaboration_rate(profile)


# Run the Analysis
instagram_analysis()
